# Laboratorio 1

## Integrantes

- Abby Donis
- Cindy Gualim
- Josué Say - 22801

## Repositorio

- [Enlace a GitHub](https://github.com/JosueSay/labs-ms/tree/main/Labs/lab1)

## Preparación de entorno

Usar si se está trabajando en lab:

In [ ]:
using Pkg
Pkg.activate(".")

Pkg.add("JuMP")
Pkg.add("JuMP")
Pkg.add("HiGHS")
Pkg.add("Ipopt")
Pkg.add("Optimization")

# Instalar otros paquetes necesarios...

Pkg.activate()
Pkg.status()

Usar si ya se ha terminado el lab:

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
Pkg.status()

# Problema 3 - Modelo de asignación de horarios

La ciudad de Guatemala estudia la factibilidad de utilizar un sistema de autobuses de transporte masivo para reducir el tráfico urbano. El estudio busca la cantidad mínima de autobuses que satisfaga las necesidades de transporte. Después de reunir la información necesaria, el ingeniero de tránsito observó que la cantidad mínima de autobuses que se requería fluctuaba según la hora del día, y dicha cantidad se podía representar de forma aproximada por valores constantes durante intervalos de 4 horas sucesivos. La siguiente figura resume los hallazgos del ingeniero.

| Turno | Horario                     |
|-------|-----------------------------|
| 1     | 00:00 A.M. a 07:59 A.M.     |
| 2     | 04:00 A.M. a 11:59 A.M.     |
| 3     | 08:00 A.M. a 03:59 P.M.     |
| 4     | 12:00 P.M. a 07:59 P.M.     |
| 5     | 04:00 P.M. a 11:59 P.M.     |
| 6     | 08:00 P.M. a 03:59 A.M.     |

!["Cantidad (demanda) de buses en función de la hora del día"](./images/ejemplo_p3.png)

Para realizar el mantenimiento diario requerido, cada autobús puede operar sólo 8 horas continuas al día. Aquí, $x_i$ es la cantidad de autobuses que inician en el turno *i*. En este problema, se desea minimizar el número total de buses circulantes diarios, de modo que en cada tramo de tiempo, satisfaga la distribución de la demanda requerida (ver figura).

## Inciso a

Formular el problema de programación lineal

### Contexto

Se quiere minimizar el número total de autobuses necesarios para satisfacer la demanda durante el día, dividiendo el día en seis turnos de 8 horas. Cada autobús trabaja una jornada continua de 8 horas (por mantenimiento), iniciando en uno de los seis turnos definidos.

La demanda de autobuses varía por tramos de 4 horas a lo largo del día (como se ve en la parte superior del gráfico). Para cada tramo de 4 horas se requiere una cierta cantidad mínima de autobuses circulando.

Lo que se debe hacer es poder asignar la cantidad adecuada de buses a cada turno para cubrir todos los horarios sin poner buses de más.

#### Turnos y Horarios

| Turno | Horario       |
| ----- | ------------- |
| 1     | 00:00 – 07:59 |
| 2     | 04:00 – 11:59 |
| 3     | 08:00 – 15:59 |
| 4     | 12:00 – 19:59 |
| 5     | 16:00 – 23:59 |
| 6     | 20:00 – 03:59 |

Cada turno dura **8 horas** y se **traslapan** con otros turnos. Esto significa que **un mismo autobús puede cubrir varios bloques horarios** (como 04:00–07:59 + 08:00–11:59), dependiendo del turno en que inicia.

#### Imágen

| Bloque horario (4h) | Intervalo     | Demanda mínima |
| ------------------- | ------------- | -------------- |
| B1                  | 00:00 – 03:59 | 4              |
| B2                  | 04:00 – 07:59 | 8              |
| B3                  | 08:00 – 11:59 | 10             |
| B4                  | 12:00 – 15:59 | 7              |
| B5                  | 16:00 – 19:59 | 12             |
| B6                  | 20:00 – 23:59 | 4              |

**Cada bloque horario debe tener al menos esa cantidad de autobuses operando, sin importar en qué turno empezaron.**

Cada turno cubre 8h continuas:

* **B1** (00:00–03:59): lo cubren $x_1$ y $x_6$
* **B2** (04:00–07:59): lo cubren $x_1$ y $x_2$
* **B3** (08:00–11:59): lo cubren $x_2$ y $x_3$
* **B4** (12:00–15:59): lo cubren $x_3$ y $x_4$
* **B5** (16:00–19:59): lo cubren $x_4$ y $x_5$
* **B6** (20:00–23:59): lo cubren $x_5$ y $x_6$

### Formulación del problema

#### Variable

$x_i$ = cantidad de autobuses que inician turno $i$, con $i = 1,\dots,6$

*Esto lo sabemos dado que cada bus solo opera 8 horas (un turno). Si sabemos cuántos inician en cada turno, podemos calcular cuántos hay operando en cada bloque de 4 horas.*

#### Función objetivo

Minimizar la cantidad total de autobuses operando:

$$
\min z = x_1 + x_2 + x_3 + x_4 + x_5 + x_6
$$

*Esto directamente nos lo piden ya que menos autobuses significa menos costos de operación, mantenimiento y personal, siempre que se cumpla con la demanda de transporte en cada bloque.*

### Restricciones por bloque de horario

$$
\begin{aligned}
x_1 + x_6 &\geq 4 \quad \text{(Bloque 1: 12:00am – 4:00am)} \\
x_1 + x_2 &\geq 8 \quad \text{(Bloque 2: 4:00am – 8:00am)} \\
x_2 + x_3 &\geq 10 \quad \text{(Bloque 3: 8:00am – 12:00pm)} \\
x_3 + x_4 &\geq 7 \quad \text{(Bloque 4: 12:00pm – 4:00pm)} \\
x_4 + x_5 &\geq 12 \quad \text{(Bloque 5: 4:00pm – 8:00pm)} \\
x_5 + x_6 &\geq 4 \quad \text{(Bloque 6: 8:00pm – 12:00am)}
\end{aligned}
$$


*Esto refleja que los turnos que estan operando durante su bloque de 4 horas.*

### Restricción de no negatividad

$$
x_1, x_2, x_3, x_4, x_5, x_6 \in \mathbb{Z}_{\geq 0}
$$

*No se puede tener fracción de buses ni cantidades negativas*

## Inciso b

Resolver el problema usando la librería JuMP o Pulp, en variables continuas, y determinar la distribución óptima.

### Librerías

In [ ]:
using JuMP
using HiGHS

### Creación del modelo

In [ ]:
model_p3 = Model()

### Definición de variables

In [ ]:
@variable(model_p3, x1 >= 0, Int)
@variable(model_p3, x2 >= 0, Int)
@variable(model_p3, x3 >= 0, Int)
@variable(model_p3, x4 >= 0, Int)
@variable(model_p3, x5 >= 0, Int)
@variable(model_p3, x6 >= 0, Int)

### Definición de restricciones

In [ ]:
@constraint(model_p3, x1 + x6 >= 4)
@constraint(model_p3, x1 + x2 >= 8)
@constraint(model_p3, x2 + x3 >= 10)
@constraint(model_p3, x3 + x4 >= 7)
@constraint(model_p3, x4 + x5 >= 12)
@constraint(model_p3, x5 + x6 >= 4)

### Función objetivo

In [ ]:
@objective(model_p3, Min, x1 + x2 + x3 + x4 + x5 + x6)

### Configuración y aplicación del solver

In [ ]:
set_optimizer(model_p3, HiGHS.Optimizer)
optimize!(model_p3)

### Resultados

In [ ]:
println("Resultados:")
println("x1 (00:00-08:00) = ", value(x1))
println("x2 (04:00-12:00) = ", value(x2))
println("x3 (08:00-16:00) = ", value(x3))
println("x4 (12:00-20:00) = ", value(x4))
println("x5 (16:00-24:00) = ", value(x5))
println("x6 (20:00-04:00) = ", value(x6))
println("Total mínimo de autobuses necesarios (z) = ", objective_value(model_p3))

### Respuesta

| Turno    | Horario       | $x_i$    | Resultado                                       |
| -------- | ------------- | -------- | ----------------------------------------------- |
| $x_1$    | 00:00 – 08:00 | 4        | Inician 4 buses a medianoche                    |
| $x_2$    | 04:00 – 12:00 | 8        | Inician 8 buses a las 4 am                      |
| $x_3$    | 08:00 – 16:00 | 2        | Inician 2 buses a las 8 am                      | 
| $x_4$    | 12:00 – 20:00 | 7        | Inician 7 buses a mediodía                      |
| $x_5$    | 16:00 – 24:00 | 5        | Inician 5 buses a las 4 pm                      |
| $x_6$    | 20:00 – 04:00 | 0        | No se necesita ningún bus comenzando a las 8 pm |

En los resultados podemos ver cuántos buses deben iniciar su turno en dicha hora para los $x_i$, dando una suma total de 26 buses al día para cubrir la demanda.

# Problema 4 - Modelo de renovación urbana

La ciudad de Erstville enfrenta un grave recorte de presupuesto. Buscando una solución a largo plazo para mejorar la base tributaria, el consejo de la ciudad propone la demolición de un área de viviendas dentro de la ciudad, y su reemplazo con un moderno desarrollo. El proyecto implica dos fases: (1) demolición de casas populares para obtener el terreno para el nuevo desarrollo, y (2) construcción del nuevo desarrollo. A continuación, un resumen de la situación.

1. Se pueden demoler 300 casas populares. Cada casa ocupa un lote de 0.25 acres. El costo de demoler una casa es de $2000.  
2. Los tamaños de los lotes para construir casas unifamiliares, dobles, triples y cuádruples, son de 0.18, 0.28, 0.4 y 0.5 acres, respectivamente. Las calles, los espacios abiertos y el área para la instalación de servicios, ocupan 15% del área disponible.  
3. En el nuevo desarrollo, las unidades triples y cuádruples ocupan por lo menos 25% del total. Las unidades sencillas deben ser al menos 20% de todas las unidades, y las unidades dobles deben ocupar un mínimo de 10%.  
4. El impuesto por unidad aplicado a las unidades sencillas, dobles, triples y cuádruples es de $1000, $1900, $2700 y $3400, respectivamente.  
5. El costo de construcción por unidad de las casas sencillas, dobles, triples y cuádruples es de $50,000, $70,000, $130,000 y $160,000, respectivamente. El financiamiento a través de un banco local está limitado a $15 millones.

**¿Cuántas unidades de cada tipo se deben construir para maximizar la recaudación de impuestos? Deberá hacer lo siguiente:**

## Inciso a

Formular el problema de programación lineal.

### Contexto

Se quiere maximizar la recaudación de impuestos a futuro sujeto a la nueva construcción de casas considerando las condiciones de espacio, presupuesto y porcentaje de tipo de unidad de casas.

#### Datos

##### Demoliciones

- Se pueden demoler máximo 300 casas.
- Cada una ocupa 0.25 acres
  - Por lo tanto el total máximo del terreno disponible es de $300 \times 0.25 = 75$ acres
- Costo por demolición por casa: \$2,000
  - Por lo tanto el total es de $300\times\text{\$}2000=\text{\$}600,000$


##### Tipos de viviendas nuevas

| Tipo      | Acre por unidad | Impuesto (\$) | Costo de construcción (\$) |
| --------- | --------------- | ------------- | -------------------------- |
| Sencilla  | 0.18            | 1000          | 50,000                     |
| Doble     | 0.28            | 1900          | 70,000                     |
| Triple    | 0.40            | 2700          | 130,000                    |
| Cuádruple | 0.50            | 3400          | 160,000                    |

El **15%** del área disponible se usará para otro motivo que no son las casas, por lo tanto tenemos un **85%** del aréa que puede usarse por lo tanto realmente se tiene un área disponible de:

$$0.85 \times 75 = 63.75\text{ acres}$$

##### Otras restricciones

- Financiamiento máximo: \$15,000,000
- Porcentaje mínimo por tipo de unidad:

  | Tipo de unidad                  | Porcentaje mínimo requerido |
  | ------------------------------- | --------------------------- |
  | Sencilla                        | 20% del total de unidades   |
  | Doble                           | 10% del total de unidades   |
  | Triple + Cuádruple (combinadas) | 25% del total de unidades   |


### Formulación del problema

#### Variable

Definimos las variables de acuerdo al tipo de unidad de casa:

* $x_1$ = número de **unidades sencillas**
* $x_2$ = número de **unidades dobles**
* $x_3$ = número de **unidades triples**
* $x_4$ = número de **unidades cuádruples**

*Esto es porque debemos determinar cuantas unidades construir de cada tipo.*

#### Función objetivo

Maximizar la recaudación de impuestos:

$$
\max z = 1000x_1 + 1900x_2 + 2700x_3 + 3400x_4
$$

*Esto debido a que cada tipo de unidad de casa genera impuesto y queremos maximizar esa suma total.*

### Restricciones de espacio

Ya que tenemos de área disponible $63.75$ acres y tenemos definido que cada espacio consume un porcentaje:

$$
0.18x_1 + 0.28x_2 + 0.4x_3 + 0.5x_4 \leq 63.75
$$

*Esto quiere decir que el área ocupada por todas las viviendas no debe exceder el área disponible*

### Restricciones de presupuesto

$$
50000x_1 + 70000x_2 + 130000x_3 + 160000x_4 \leq 15,000,000
$$


*Esto quiere decir que no debemos pasarnos del presupuesto dado.*

### Restricciones por tipo de unidad de casa

1. **Al menos el 25% de las casas deben ser triples o cuadrúples:**

  $$
  x_3 + x_4 \geq 0.25(x_1 + x_2 + x_3 + x_4)
  $$

  $$
  4(x_3 + x_4) \geq x_1 + x_2 + x_3 + x_4
  $$

2. **Al menos el 20% de las casas deben ser sencillas:**

  $$
  x_1 \geq 0.2(x_1 + x_2 + x_3 + x_4)
  $$

  $$
  5x_1 \geq x_1 + x_2 + x_3 + x_4
  $$

3. **Al menos el 10% de las casas deben ser dobles:**

  $$
  x_2 \geq 0.1(x_1 + x_2 + x_3 + x_4)
  $$

  $$
  10x_2 \geq x_1 + x_2 + x_3 + x_4
  $$

### Restricción de no negatividad

$$
x_1, x_2, x_3, x_4 \in \mathbb{Z}_{\geq 0}
$$

*No se puede tener fracción de buses ni cantidades negativas*

## Inciso b

Resolver el problema usando la librería JuMP o Pulp, en variables continuas, y determinar la distribución óptima.  

**Función objetivo:**

$$
\max z = 1000x_1 + 1900x_2 + 2700x_3 + 3400x_4
$$

**Sujeto a:**

$$
\begin{aligned}
0.18x_1 + 0.28x_2 + 0.4x_3 + 0.5x_4 &\leq 63.75 \quad \text{(terreno)} \\
50000x_1 + 70000x_2 + 130000x_3 + 160000x_4 &\leq 15,000,000 \quad \text{(presupuesto)} \\
4(x_3 + x_4) &\geq x_1 + x_2 + x_3 + x_4 \quad \text{(mínimo 25\% triples/cuádruples)} \\
5x_1 &\geq x_1 + x_2 + x_3 + x_4 \quad \text{(mínimo 20\% sencillas)} \\
10x_2 &\geq x_1 + x_2 + x_3 + x_4 \quad \text{(mínimo 10\% dobles)} \\
x_1, x_2, x_3, x_4 \geq 0
\end{aligned}
$$


### Librerías

In [ ]:
using JuMP
using HiGHS

### Creación del modelo

In [ ]:
model_4b = Model()

### Definición de variables

In [ ]:
@variable(model_4b, x1 >= 0)
@variable(model_4b, x2 >= 0)
@variable(model_4b, x3 >= 0)
@variable(model_4b, x4 >= 0)

### Definición de restricciones

In [ ]:
# terreno
@constraint(model_4b, 0.18x1 + 0.28x2 + 0.4x3 + 0.5x4 <= 63.75)

# presupuesto
@constraint(model_4b, 50000x1 + 70000x2 + 130000x3 + 160000x4 <= 15_000_000)

# tipo de unidad de casa
@constraint(model_4b, 4(x3 + x4) >= x1 + x2 + x3 + x4)
@constraint(model_4b, 5x1 >= x1 + x2 + x3 + x4)
@constraint(model_4b, 10x2 >= x1 + x2 + x3 + x4)

### Función objetivo

In [ ]:
@objective(model_4b, Max, 1000x1 + 1900x2 + 2700x3 + 3400x4)

### Configuración y aplicación del solver

In [ ]:
set_optimizer(model_4b, HiGHS.Optimizer)
optimize!(model_4b)

### Resultados

In [ ]:
println("x1 (sencillas)   = ", value(x1))
println("x2 (dobles)      = ", value(x2))
println("x3 (triples)     = ", value(x3))
println("x4 (cuádruples)  = ", value(x4))
println("Total impuesto (z) = ", objective_value(model_4b))

## Inciso c

Resolver el problema en variables enteras, y comparar la solución con la de (b). Discuta sus resultados.

**Función objetivo:**

$$
\max z = 1000x_1 + 1900x_2 + 2700x_3 + 3400x_4
$$

**Sujeto a:**

$$
\begin{aligned}
0.18x_1 + 0.28x_2 + 0.4x_3 + 0.5x_4 &\leq 63.75 \quad \text{(terreno)} \\
50000x_1 + 70000x_2 + 130000x_3 + 160000x_4 &\leq 15,000,000 \quad \text{(presupuesto)} \\
4(x_3 + x_4) &\geq x_1 + x_2 + x_3 + x_4 \quad \text{(mínimo 25\% triples/cuádruples)} \\
5x_1 &\geq x_1 + x_2 + x_3 + x_4 \quad \text{(mínimo 20\% sencillas)} \\
10x_2 &\geq x_1 + x_2 + x_3 + x_4 \quad \text{(mínimo 10\% dobles)} \\
x_1, x_2, x_3, x_4 &\in \mathbb{Z}_{\geq 0}
\end{aligned}
$$


### Librerías

In [ ]:
using JuMP
using HiGHS

### Creación del modelo

In [ ]:
model_4c = Model()

### Definición de variables

In [ ]:
@variable(model_4c, x1 >= 0, Int)
@variable(model_4c, x2 >= 0, Int)
@variable(model_4c, x3 >= 0, Int)
@variable(model_4c, x4 >= 0, Int)

### Definición de restricciones

In [ ]:
# terreno
@constraint(model_4c, 0.18x1 + 0.28x2 + 0.4x3 + 0.5x4 <= 63.75)

# presupuesto
@constraint(model_4c, 50000x1 + 70000x2 + 130000x3 + 160000x4 <= 15_000_000)

# tipo de unidad de casa
@constraint(model_4c, 4(x3 + x4) >= x1 + x2 + x3 + x4)
@constraint(model_4c, 5x1 >= x1 + x2 + x3 + x4)
@constraint(model_4c, 10x2 >= x1 + x2 + x3 + x4)

### Función objetivo

In [ ]:
@objective(model_4c, Max, 1000x1 + 1900x2 + 2700x3 + 3400x4)

### Configuración y aplicación del solver

In [ ]:
set_optimizer(model_4c, HiGHS.Optimizer)
optimize!(model_4c)

### Resultados

In [ ]:
println("x1 (sencillas)   = ", value(x1))
println("x2 (dobles)      = ", value(x2))
println("x3 (triples)     = ", value(x3))
println("x4 (cuádruples)  = ", value(x4))
println("Total impuesto (z) = ", objective_value(model_4c))

### Discusión de resultados

| Tipo de unidad        | Variables continuas | Variables enteras |
| --------------------- | ------------------- | ----------------- |
| Sencillas ($x_1$)     | 37.04               | 36                |
| Dobles ($x_2$)        | 101.85              | 99                |
| Triples ($x_3$)       | 46.30               | 31                |
| Cuádruples ($x_4$)    | 0                   | 14                |
| **Impuesto total**    | **355,556**         | **355,400**       |


Sabemos que idealmente las restricciones para el inciso c son mejores por el hecho que la construcción de casas deben ser enteras porque las fracciones no pueden ser vendidas pero ademas tomemos algo adicional en cuenta, si tomamos los resultados continuos como solución teórica y los resultados enteros como solución verdadera podemos estimar incluso una pérdida asociada a construir hacer ese sacrificio de construir casas enteras a comparación de continuas:

$$\text{perdida}= \frac{355556 - 355400}{355556} ≈ 0.044\%$$

Si bien se tiene una menor ganancia no es tanta y se puede asumir el "riesgo" de idealmente construir casas completas en lugar de fracciones.
